In [28]:
# Import Dep
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time# Imports and requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from citipy import citipy
from datetime import datetime
from scipy.stats import linregress
#Import the requests library
import requests
#Import the API key.
from config import weather_api_key

In [29]:
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [30]:
coordinates = list(lat_lngs)

In [31]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

756

In [32]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [41]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description
                        })

# If an error is experienced, skip the city.
    except KeyError:
            print("City not found. Skipping...")
            pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | busselton
Processing Record 2 of Set 1 | wewak
Processing Record 3 of Set 1 | belmonte
Processing Record 4 of Set 1 | mataura
Processing Record 5 of Set 1 | tambopata
Processing Record 6 of Set 1 | aksu
Processing Record 7 of Set 1 | victoria
Processing Record 8 of Set 1 | vaini
Processing Record 9 of Set 1 | saquarema
Processing Record 10 of Set 1 | kapaa
Processing Record 11 of Set 1 | atuona
Processing Record 12 of Set 1 | punta arenas
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | sena madureira
Processing Record 15 of Set 1 | lavrentiya
Processing Record 16 of Set 1 | xingyi
Processing Record 17 of Set 1 | saint george
Processing Record 18 of Set 1 | oeiras do para
Processing Record 19 of Set 1 | torbay
Processing Record 20 of Set 1 | qaanaaq
Processing Record 21 of Set 1 | kodiak
Processing Record 22 of Set 1 | airai
Processing Record 23 of Set 1 | nishihara
Proc

In [42]:
len(city_data)

701

In [43]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.65,115.33,64.99,65,97,3.96,light rain
1,Wewak,PG,-3.55,143.63,85.23,68,73,6.15,light rain
2,Belmonte,BR,-15.86,-38.88,66.20,93,75,8.05,light rain
3,Mataura,NZ,-46.19,168.86,55.99,68,25,10.00,scattered clouds
4,Tambopata,PE,-12.73,-69.18,69.67,60,32,2.66,scattered clouds
5,Aksu,CN,41.12,80.26,51.85,39,1,1.32,clear sky
6,Victoria,HK,22.29,114.16,84.99,55,86,1.99,overcast clouds
7,Vaini,TO,-21.20,-175.20,78.80,83,40,10.29,smoke
8,Saquarema,BR,-22.92,-42.51,64.74,77,16,10.40,few clouds
9,Kapaa,US,22.08,-159.32,73.99,100,90,9.17,light rain


In [44]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_DataBase.csv"
city_data_df.to_csv(output_data_file, index_label="City_ID")